**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-sbtsivrt
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-sbtsivrt
  Resolved https://github.com/Kaggle/learntools.git to commit 0b915896f0a90bedb3b94d36398a53cd31c402d7
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268968 sha256=9401e7455862885ba6d504b3281eb3b7fe58e998b82cff0ea2bf1069e7e1d5f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-de503tx7/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,bc0e1d5075c00f71c1102b193c64e348cd775a97,cb3bd991c2f7932527ef9d5f341674dfba53745f25fe3e...,2017-12-08 10:15:00+00:00,2017-12-08 10:15:00+00:00,0,0.0,NaN,NaN,13,13,...,0.0,3.25,Cash,Blue Ribbon Taxi Association Inc.,41.983636,-87.723583,POINT (-87.7235831853 41.9836363072),41.983636,-87.723583,POINT (-87.7235831853 41.9836363072)
1,ffe8378751e25d68284444ceb330480a0aa1bc40,c7128929398a2f461a7ad7c28638c2719c4fabb7c3e708...,2017-12-08 14:30:00+00:00,2017-12-08 14:30:00+00:00,0,0.0,NaN,NaN,59,59,...,1.0,4.25,Cash,Taxi Affiliation Services,41.829922,-87.672503,POINT (-87.6725026464 41.8299223039),41.829922,-87.672503,POINT (-87.6725026464 41.8299223039)
2,07616d0456bc3380463d96f02af18ad21d4aff78,aafb86c1bc07cf16a407a8229564ff200f0ff2e8ca2b56...,2017-12-28 12:30:00+00:00,2017-12-28 12:30:00+00:00,0,0.0,1.703184e+10,1.703184e+10,59,59,...,0.0,3.25,Cash,KOAM Taxi Association,41.833518,-87.681356,POINT (-87.6813558293 41.8335178865),41.833518,-87.681356,POINT (-87.6813558293 41.8335178865)
3,a50ceabd0ab53d220ecee073265a7b04c2ea1b04,9ff153898c75dbd960a207b660a71f92f79b666dcec98e...,2017-12-09 10:15:00+00:00,2017-12-09 10:30:00+00:00,1380,8.9,NaN,NaN,42,54,...,0.0,26.00,Cash,Taxi Affiliation Services,41.778877,-87.594925,POINT (-87.5949254391 41.7788768603),41.660136,-87.602848,POINT (-87.60284764 41.6601360507)
4,5affa071d286562a229abc8bc0ffadb9b921ebec,cc5330b266a2b3e042e5bc50b1dadb4f4e03db62f8cbd1...,2017-12-18 01:00:00+00:00,2017-12-18 01:00:00+00:00,360,1.5,NaN,NaN,1,2,...,0.0,6.75,Cash,Taxi Affiliation Services,42.009623,-87.670167,POINT (-87.67016685690001 42.0096228806),42.001571,-87.695013,POINT (-87.6950125892 42.001571027)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-02-17,90092.857143
1,2016-01-06,74708.571429
2,2016-01-19,85156.571429
3,2016-02-05,81493.142857
4,2016-01-20,84509.142857


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() OVER (
                        PARTITION BY pickup_community_area
                        ORDER BY trip_start_timestamp                       
                        ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,3.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
1,3.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
2,3.0,2013-10-03 00:00:00+00:00,2013-10-03 00:30:00+00:00,1
3,3.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
4,3.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,0f1b65c1c73b1a8a687054ea0bb175edd243a5d065d7fa...,2013-10-03 15:45:00+00:00,2013-10-03 16:00:00+00:00,180.0
1,19fbaf2c91768dbdf4d25436a17f9f99da4330f1af0d66...,2013-10-03 13:30:00+00:00,2013-10-03 13:45:00+00:00,225.0
2,1b106798d36c2a85aa61c47ef4a331e1c3f00d159601ce...,2013-10-03 18:30:00+00:00,2013-10-03 18:45:00+00:00,255.0
3,30b02068f1c857f6fd5144c600f3e61acb5f5456e5fb0e...,2013-10-03 07:00:00+00:00,2013-10-03 07:00:00+00:00,210.0
4,35467b44491f6f51eaa0f4fb1cd65e4c23117aa268d9dd...,2013-10-03 16:00:00+00:00,2013-10-03 16:00:00+00:00,-45.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*